<center>
  <p style="text-align:center">
    <img
      alt="hypertion logo"
      height="120"
      src="https://raw.githubusercontent.com/synacktraa/hypertion/master/assets/main.gif"
    >
    <br>
    <a href="https://github.com/synacktraa/hypertion">Github</a>
    |
    <a href="https://pypi.org/project/hypertion">PyPI</a>
    |
    <a href="https://drive.google.com/file/d/1y0hf-8leMnk0fnTPY9FWnCgu3ePJqx0G/view?usp=sharing">Colab</a>
  </p>
</center>
<h1 align="center">Automating Function Calling Workflow</h1>

In this tutorial, you will:

- Use Hypertion to register functions and automate creation of Mistral specific function-calling schema.
- Utilize Mistral's [function calling feature](https://docs.mistral.ai/guides/function-calling/) to obtain function-call metadata(function name and raw arguments).
- Compose a function-call object from Mistral's metadata.

## Background

One powerful feature of the Mistral AI chat completions API is tool calling, wherein a user describes the signature and arguments of one or more tools to the Mistral AI API via a JSON Schema and natural language descriptions, and the LLM decides when to use each tool and provides argument values depending on the context of the conversation.

Creating JSON schema from functions and invoking those functions from LLM's output manually can be time-consuming and potentially complex, depending on the function parameters. Hypertion takes responsibility of preparing inputs for the LLM, validating outputs from the LLMs' output and finally constructing a function-call object which can be inspected or invoked.

Supported parameter types: `str` | `int` | `float` | `bool` | `list` | `dict` | `pathlib.Path` | `typing.List` | `typing.Dict` | `typing.NamedTuple` | `typing_extensions.TypedDict` | `pydantic.BaseModel` | `typing.Literal` | `enum.Enum`

Let's get started!

ℹ️ This notebook requires a Mistral AI API key.


## 1. Setup

Install dependencies.

In [1]:
%pip install -Uq mistralai hypertion pydantic duckduckgo_search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00


Import libraries.

In [2]:
import os
from getpass import getpass

from hypertion import HyperFunction
from mistralai.client import MistralClient

Set your Mistral API key and create instances of `MistralClient` and `Hyperfunction`

In [3]:
if not (api_key := os.getenv("MISTRAL_API_KEY")):
    api_key = getpass("🔑 Enter your Mistral AI API key: ")

client = MistralClient(api_key=api_key)
hyperfunction = HyperFunction()

🔑 Enter your Mistral AI API key: ··········


## 2. Define function and register it

We're defining a function with parameters types that are nested and of object type to showcase hypertion's ability to create schema.

In [4]:
import pathlib
from enum import Enum
from typing import Literal, NamedTuple
from typing_extensions import TypedDict
from pydantic import BaseModel, Field


class ServerType(str, Enum):
    TESTING = "testing"
    DEVELOPMENT = "development"
    PRODUCTION = "production"

class ServerConfig(BaseModel):
    host: str = Field(default="localhost", description="The host url of the server.")
    port: int = Field(..., description="The port of the server.")
    use_ssl: bool = False

class LoggingConfig(TypedDict):
    """
    Logging Configuration
    :param level: Level of the logging.
    :param format: Logging format.
    """
    level: Literal['debug', 'info', 'warning', 'error', 'critical'] = 'info'
    format: str

class User(NamedTuple):
    """
    User Info
    :param name: Name of the user.
    :param role: Role of the user.
    """
    name: str
    role: Literal['admin', 'developer', 'tester'] = 'developer'

@hyperfunction.takeover
def setup_server(
    name: str,
    server_type: ServerType,
    server_config: ServerConfig,
    log_config: LoggingConfig,
    users: list[User],
    persist_at: pathlib.Path,
) -> None:
    """
    Setup server with the given configuration.

    @param name: The name of the server.
    @param server_type: The type of the server.
    @param server_config: Configuration for the server.
    @param log_config: Logging configuration.
    @param users: List of users with their respective roles.
    @param persist_at: Path to persist server log.
    """
    print(f"Setting up server '{name}' of type '{server_type}'...")
    print(f"Server Configuration: {server_config}")
    print(f"Logging Configuration: {log_config}")
    print(f"Users: {users}")
    print(f"Persist location: {persist_at}")

Display registered functions.

In [5]:
hyperfunction.registry()

setup_server(
   name: str,
   server_type: ServerType,
   server_config: ServerConfig,
   log_config: LoggingConfig,
   users: list[User],
   persist_at: Path
) -> None:
"""Setup server with the given configuration."""


## 3. Construct `Mistral` specific function schema

In [6]:
print(hyperfunction.format('mistral', as_json=True))

[
    {
        "type": "function",
        "function": {
            "name": "setup_server",
            "description": "Setup server with the given configuration.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "The name of the server."
                    },
                    "server_type": {
                        "type": "string",
                        "enum": [
                            "TESTING",
                            "DEVELOPMENT",
                            "PRODUCTION"
                        ],
                        "description": "The type of the server."
                    },
                    "server_config": {
                        "type": "object",
                        "properties": {
                            "host": {
                                "type": "string",
                   

## 4. Request `Mistral` metadata and compose function-call

In [7]:
from mistralai.models.chat_completion import ChatMessage, ToolChoice

def request_mistral_metadata(query: str, functions: list[dict]) -> tuple[str, str]:
    tool_call = client.chat(
        model="mistral-large-latest",
        messages=[ChatMessage(role='user', content=query)],
        tools=functions,
        tool_choice=ToolChoice.any
    ).choices[0].message.tool_calls[0].function
    return tool_call.name, tool_call.arguments # We don't have to manually convert it to dictionary object, hyperfunction supports both json and dictionary object.

In [8]:
name, arguments = request_mistral_metadata(query="""\
Setup a server of type development on port number 8001 with ssl enabled.
Logging format should be "%(asctime)s - %(name)s - %(levelname)s - %(message)s".
Add user sophia as admin, synacktra as developer and 1littlecoder as tester.
The server logs should be persisted at /var/logs.\
""", functions=hyperfunction.format('mistral'))
print(f"Function: {name}\nRaw Arguments: {arguments}")

Function: setup_server
Raw Arguments: {"name": "my_server", "server_type": "DEVELOPMENT", "server_config": {"port": 8001, "use_ssl": true}, "log_config": {"format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s"}, "users": [{"name": "sophia", "role": "admin"}, {"name": "synacktra", "role": "developer"}, {"name": "1littlecoder", "role": "tester"}], "persist_at": "/var/logs"}


Compose the function to get function-call object.

In [9]:
function_call = hyperfunction.compose(name=name, arguments=arguments)
function_call

setup_server(
   name='my_server',
   server_type=<ServerType.DEVELOPMENT: 'development'>,
   server_config=ServerConfig(host='localhost', port=8001, use_ssl=True),
   log_config={'level': 'info', 'format': '%(asctime)s - %(name)s - %(levelname)s - %(message)s'},
   users=[('sophia', 'admin'), ('synacktra', 'developer'), ('1littlecoder', 'tester')],
   persist_at=PosixPath('/var/logs')
)

> Hyperfunction performs parameter validation, type conversion, and applies default values for function parameters not available in LLM generated arguments.

Inspect the parameters.

In [10]:
function_call.arguments

{'name': 'my_server',
 'server_type': <ServerType.DEVELOPMENT: 'development'>,
 'server_config': ServerConfig(host='localhost', port=8001, use_ssl=True),
 'log_config': {'level': 'info',
  'format': '%(asctime)s - %(name)s - %(levelname)s - %(message)s'},
 'users': [('sophia', 'admin'),
  ('synacktra', 'developer'),
  ('1littlecoder', 'tester')],
 'persist_at': PosixPath('/var/logs')}

Invoke the function.

In [11]:
function_call()

Setting up server 'my_server' of type 'development'...
Server Configuration: host='localhost' port=8001 use_ssl=True
Logging Configuration: {'level': 'info', 'format': '%(asctime)s - %(name)s - %(levelname)s - %(message)s'}
Users: [('sophia', 'admin'), ('synacktra', 'developer'), ('1littlecoder', 'tester')]
Persist location: /var/logs


---

## Create a new `HyperFunction` instance for search task

Here we are creating a new instance for storing search related functions, using instances we can manage distinct set of tools.

In [16]:
from duckduckgo_search import DDGS

ddgs = DDGS()
search_task = HyperFunction()

@search_task.takeover
def search_text(
  text: str,
  *, # Yes it supports positional and keyword arguments
  safe_search: bool = True,
  backend: Literal['api', 'html', 'lite'] = 'api',
  max_results: int = 1
):
    """
    Search for text in the web.
    :param text: Text to search for.
    :param safe_search: If True, enable safe search.
    :param backend: Backend to use for retrieving results.
    :param max_results: Max results to return.
    """
    return ddgs.text(
        keywords=text,
        safesearch="on" if safe_search else "off",
        backend=backend,
        max_results=max_results
    )

@search_task.takeover
def search_image(
  query: str,
  *,
  size: Literal['Small', 'Medium', 'Large', 'Wallpaper'] = 'Small',
  layout: Literal['Square', 'Tall', 'Wide'] = 'Square',
  safe_search: bool = True,
  max_results: int = 1
):
    """
    Search for images in the web.
    :param query: Query to search for.
    :param safe_search: If True, enable safe search.
    :param max_results: Max results to return.
    """
    return ddgs.images(
        keywords=query,
        size=size,
        layout=layout,
        safesearch="on" if safe_search else "off",
        max_results=max_results
    )

@search_task.takeover
def translate(
    text: str,
    to: Literal['en', 'ja', 'hi', 'es', 'fr', 'de', 'zh'] = 'en'
):
    """
    Translate the given text.
    :param text: Text to translate.
    :param to: what language to translate.
    """
    return ddgs.translate(keywords=text, to=to)


Store function schemas in a variable.

In [17]:
search_function_schemas = search_task.format('mistral')

#### Searching for text.

In [18]:
name, arguments = request_mistral_metadata(
    query="What is Mistral Function Calling?", functions=search_function_schemas
)
name, arguments

('search_text', '{"text": "Mistral Function Calling", "max_results": 3}')

Compose function call object.

In [19]:
search_text_call = search_task.compose(name=name, arguments=arguments)
search_text_call

search_text(
   text='Mistral Function Calling',
   safe_search=True,
   backend='api',
   max_results=3
)

In [20]:
search_text_call()

[{'title': 'Function calling | Mistral AI Large Language Models',
  'href': 'https://docs.mistral.ai/capabilities/function_calling/',
  'body': 'Function calling allows Mistral models to connect to external tools. By integrating Mistral models with external tools such as user defined functions or APIs, users can easily build applications catering to specific use cases and practical problems. In this guide, for instance, we wrote two functions for tracking payment status and payment date.'},
 {'title': 'Mistral AI Function Calling — A Simple Example with Code',
  'href': 'https://medium.com/@jcrsch/mistral-ai-function-calling-a-simple-example-with-code-72f3a762550f',
  'body': 'Learn how to use the function calling API of Mistral AI, a platform for creating assistants capable of actions (LAM), with a home automation use case. See the code, the functions, and the natural language description of the functions and their parameters.'},
 {'title': 'Advanced Function Calling with Mistral-7B -

#### Searching for images.

In [21]:
name, arguments = request_mistral_metadata(
    query="Get me medium size images of Transformers architecture",
    functions=search_function_schemas
)
name, arguments

('search_image',
 '{"query": "Transformers architecture", "size": "Medium", "max_results": 10}')

In [22]:
search_image_call = search_task.compose(name=name, arguments=arguments)
search_image_call

search_image(
   query='Transformers architecture',
   size='Medium',
   layout='Square',
   safe_search=True,
   max_results=10
)

In [23]:
search_image_call()

[{'title': 'Understading Matrix Operations In Transformers',
  'image': 'https://mchromiak.github.io/articles/2017/Sep/12/Transformer-Attention-is-all-you-need/img/encoder.png',
  'thumbnail': 'https://tse4.mm.bing.net/th?id=OIP.Dtz4GnZ1uYNMIxRC5mKScgAAAA&pid=Api',
  'url': 'https://santhoshkolloju.github.io/transformers/',
  'height': 421,
  'width': 474,
  'source': 'Bing'},
 {'title': 'What is a Transformer Model? | Definition from TechTarget',
  'image': 'https://cdn.ttgtmedia.com/rms/onlineimages/transformer_model_architecture-f_mobile.png',
  'thumbnail': 'https://tse1.mm.bing.net/th?id=OIP.Wz_LKIcZudhsfzHcH6aJLAAAAA&pid=Api',
  'url': 'https://www.techtarget.com/searchenterpriseai/definition/transformer-model',
  'height': 468,
  'width': 474,
  'source': 'Bing'},
 {'title': 'The Architecture of the Transformers. | Download Scientific Diagram',
  'image': 'https://www.researchgate.net/publication/355846101/figure/fig5/AS:1085678683197443@1635857377407/The-Architecture-of-the-Tra

#### Searching for translation.

In [24]:
name, arguments = request_mistral_metadata(
    query="Translate 'Attention is all you need' to Hindi.",
    functions=search_function_schemas
)
name, arguments

('translate', '{"text": "Attention is all you need", "to": "hi"}')

In [25]:
translate_call = search_task.compose(name=name, arguments=arguments)
translate_call

translate(
   text='Attention is all you need',
   to='hi'
)

In [26]:
translate_call()

[{'detected_language': 'en',
  'translated': 'ध्यान आप सभी की जरूरत है',
  'original': 'Attention is all you need'}]

## Combine `HyperFunction` instances

> Note: A single instance can hold multiple functions. Creating a new instance is beneficial only if you need a distinct set of functions. This approach is especially effective when deploying Agent(s) to utilize functions designed for particular tasks.

In [27]:
new_instance = hyperfunction + search_task
new_instance.registry()

setup_server(
   name: str,
   server_type: ServerType,
   server_config: ServerConfig,
   log_config: LoggingConfig,
   users: list[User],
   persist_at: Path
) -> None:
"""Setup server with the given configuration."""
search_text(
   text: str,
   *,
   safe_search: bool = True,
   backend: Literal['api',
   'html',
   'lite'] = 'api',
   max_results: int = 1
):
"""Search for text in the web."""
search_image(
   query: str,
   *,
   size: Literal['Small',
   'Medium',
   'Large',
   'Wallpaper'] = 'Small',
   layout: Literal['Square',
   'Tall',
   'Wide'] = 'Square',
   safe_search: bool = True,
   max_results: int = 1
):
"""Search for images in the web."""
translate(
   text: str,
   to: Literal['en',
   'ja',
   'hi',
   'es',
   'fr',
   'de',
   'zh'] = 'en'
):
"""Translate the given text."""


In [ ]:
print(new_instance.format("mistral", as_json=True))

**This approach offers significant advantages in automating schema creation, which can reduce the time and complexity typically associated with manual setup. The `takeover` method allows for efficient management of multiple functions within a `HyperFunction` instance, which is particularly beneficial when deploying `Agents`. This automation streamlines the development process and enhances precision in handling task-specific functions, potentially improving the effectiveness of `agent-driven` scenarios.**